In [ ]:
import pysm3
import pysm3.units as u
import numpy as np
import numpy.ma as ma
import healpy as hp
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import pickle
import qubic
import os
from astropy.io import fits
from qubic import camb_interface as qc
import fgbuster as fgb
from qubicpack.utilities import Qubic_DataDir
from qubic import QubicSkySim as qss
from qubic import fibtools as ft
from qubic import camb_interface as qc
from qubic import SpectroImLib as si
from importlib import reload
from qubic import NamasterLib as nam
from comp_tools import format_alms, same_resolution
from fgbuster import get_instrument, get_sky, get_observation, ilc, basic_comp_sep, harmonic_ilc, weighted_comp_sep, multi_res_comp_sep, harmonic_ilc_alm  # Predefined instrumental and sky-creation configurations
# Imports needed for component separation
from fgbuster import (separation_recipes, xForecast, CMB, Dust, Synchrotron, FreeFree, PowerLaw,  # sky-fitting model
                      basic_comp_sep)
import ComponentSeparation

In [ ]:
### Some initializations, to be replaced with specific path, or to modify in bash
os.environ['QUBIC_DATADIR'] = '/home/mathias/Bureau/qubic/qubic'
os.environ['QUBIC_DICT'] = '/home/mathias/Bureau/qubic/qubic/dicts'
global_dir = Qubic_DataDir(datafile='instrument.py', datadir=os.environ['QUBIC_DATADIR'])

### Qubic dictionaries for 150GHz and 220Ghz
config_150, config_220 = 'FI-150', 'FI-220'
dictfilename150 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_150)
dictfilename220 = global_dir + '/doc/FastSimulator/FastSimDemo_{}.dict'.format(config_220)
d150, d220 = qubic.qubicdict.qubicDict(), qubic.qubicdict.qubicDict()
d150.read_from_file(dictfilename150)
d220.read_from_file(dictfilename220)
qub_dic = {'150': d150, '220': d220}
center = qubic.equ2gal(d220['RA_center'], d220['DEC_center'])


# Read dictionary chosen
d150['focal_length'] = 0.3
d150['nside'] = 256

#Define the number of reconstruction bands:
nbands = 5
d150['nf_recon'] = nbands
d150['nf_sub'] = nbands

# Read dictionary chosen
d220['focal_length'] = 0.3
d220['nside'] = 256

#Define the number of reconstruction bands:
d220['nf_recon'] = nbands
d220['nf_sub'] = nbands

npix = 12 * d150['nside'] ** 2
Nf = int(d150['nf_sub'])
band = d150['filter_nu'] / 1e9
filter_relative_bandwidth = d150['filter_relative_bandwidth']
npix = 12 * d220['nside'] ** 2
Nf = int(d220['nf_sub'])
band = d220['filter_nu'] / 1e9
filter_relative_bandwidth = d220['filter_relative_bandwidth']
a, nus_edge, nus_in, df, e, Nbbands_in = qubic.compute_freq(band, Nf, filter_relative_bandwidth)

def give_me_freqs_fwhm(dic, Nb) :
    band = dic['filter_nu'] / 1e9
    filter_relative_bandwidth = dic['filter_relative_bandwidth']
    a, nus_edge, nus_in, df, e, Nbbands_in = qubic.compute_freq(band, Nb, filter_relative_bandwidth)
    return nus_in, dic['synthbeam_peak150_fwhm'] * 150 / nus_in


def open_picklefile(directory, name, var) :

    """
    Open a pickle file from saved data
    """

    tab = pickle.load(open(directory + name, 'rb'))
    variable = tab[var]
    return variable

cov150 = open_picklefile('/home/mathias/Bureau/FG-Buster/', 'Coverage_dtheta_15_pointing_3000_256.pkl', 'coverage')


def give_me_mask_apo(okpix = None, lmin = 2, lmax = 512, delta_ell = 16) :
    mask = np.zeros(12 * d150['nside']**2)
    mask[okpix] = 1
    # Namaster object
    Namaster = nam.Namaster(mask, lmin=2, lmax=512, delta_ell=16)
    apo = Namaster.mask_apo
    return apo

# Define okpix
okpix_inside = (cov150 > (0.4*np.max(cov150)))
okpix = (cov150 > (0.1*np.max(cov150)))

apo = give_me_mask_apo(okpix = okpix)

okpix_fullsky = np.ones(12*d150['nside']**2)

In [ ]:
okpix_fullsky = np.ones(12*d150['nside']**2)
okpix_fullsky = okpix_fullsky != 0
okpix = cov150 != 0
okpix_inside = (cov150 > (0.5*np.max(cov150)))
nbands = 5
d150['nf_recon'] = nbands
d150['nf_sub'] = 4*nbands
d220['nf_recon'] = nbands
d220['nf_sub'] = 4*nbands

plt.figure(figsize = (16, 10))
hp.mollview(okpix_fullsky, sub = (1, 3, 1), min = 0, max = 1)
hp.mollview(okpix, sub = (1, 3, 2), min = 0, max = 1)
hp.mollview(okpix_inside, sub = (1, 3, 3), min = 0, max = 1)
plt.show()

In [ ]:
reload(ComponentSeparation)
N = 2
method = 'alms'

beta_fs_noapo_150 = np.zeros(N)

for i in range(N) :

    r_alms_fs_noapo_150, _, _ = ComponentSeparation.Apply(d150).do_comparison_pixels_alms(d150, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix_fullsky, 
                                                                   seed = None,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = False, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    beta_fs_noapo_150[i] = r_alms_fs_noapo_150.x[0]
    
print(beta_fs_noapo_150)
print(np.mean(beta_fs_noapo_150))
print(np.std(beta_fs_noapo_150))

In [ ]:
reload(ComponentSeparation)
N = 5
method = 'pixels'

beta_pixels_fs_noapo_150 = np.zeros(N)
beta_pixels_fs_noapo_220 = np.zeros(N)
beta_pixels_qs_noapo_150 = np.zeros(N)
beta_pixels_qs_noapo_220 = np.zeros(N)
beta_pixels_qs_apo_150 = np.zeros(N)
beta_pixels_qs_apo_220 = np.zeros(N)

for i in range(N) :

    r_pixels_fs_noapo_150, _, _ = ComponentSeparation.Apply(d150).do_comparison_pixels_alms(d150, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix_fullsky, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = False, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    r_pixels_fs_noapo_220, _, _ = ComponentSeparation.Apply(d220).do_comparison_pixels_alms(d220, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix_fullsky, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = False, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    r_pixels_qs_noapo_150, _, _ = ComponentSeparation.Apply(d150).do_comparison_pixels_alms(d150, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = False, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    r_pixels_qs_noapo_220, _, _ = ComponentSeparation.Apply(d220).do_comparison_pixels_alms(d220, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = False, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    r_pixels_qs_apo_150, _, _ = ComponentSeparation.Apply(d150).do_comparison_pixels_alms(d150, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = True, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    r_pixels_qs_apo_220, _, _ = ComponentSeparation.Apply(d150).do_comparison_pixels_alms(d220, 
                                                                   nbands, 
                                                                   3, 
                                                                   okpix, 
                                                                   seed = i+1,
                                                                   map1 = None, 
                                                                   map_noise = None, 
                                                                   do_apo = True, 
                                                                   noiseless = False, 
                                                                   method = method, 
                                                                   dust_only = False)
    
    beta_pixels_fs_noapo_150 = r_pixels_fs_noapo_150.x[0]
    beta_pixels_fs_noapo_220 = r_pixels_fs_noapo_220.x[0]
    beta_pixels_qs_noapo_150 = r_pixels_qs_noapo_150.x[0]
    beta_pixels_qs_noapo_220 = r_pixels_qs_noapo_220.x[0]
    beta_pixels_qs_apo_150 = r_pixels_qs_apo_150.x[0]
    beta_pixels_qs_apo_220 = r_pixels_qs_apo_220.x[0]

In [ ]:
mean_beta_pixels_fs_noapo_150 = np.mean(beta_pixels_fs_noapo_150)
mean_beta_pixels_fs_noapo_220 = np.mean(beta_pixels_fs_noapo_220)
mean_beta_pixels_qs_noapo_150 = np.mean(beta_pixels_qs_noapo_150)
mean_beta_pixels_qs_noapo_220 = np.mean(beta_pixels_qs_noapo_220)
mean_beta_pixels_qs_apo_150 = np.mean(beta_pixels_qs_apo_150)
mean_beta_pixels_qs_apo_220 = np.mean(beta_pixels_qs_apo_220)

mean_beta_alms_fs_noapo_150 = np.mean(beta_alms_fs_noapo_150)
mean_beta_alms_fs_noapo_220 = np.mean(beta_alms_fs_noapo_220)
mean_beta_alms_qs_noapo_150 = np.mean(beta_alms_qs_noapo_150)
mean_beta_alms_qs_noapo_220 = np.mean(beta_alms_qs_noapo_220)
mean_beta_alms_qs_apo_150 = np.mean(beta_alms_qs_apo_150)
mean_beta_alms_qs_apo_220 = np.mean(beta_alms_qs_apo_220)

std_beta_pixels_fs_noapo_150 = np.std(beta_pixels_fs_noapo_150)
std_beta_pixels_fs_noapo_220 = np.std(beta_pixels_fs_noapo_220)
std_beta_pixels_qs_noapo_150 = np.std(beta_pixels_qs_noapo_150)
std_beta_pixels_qs_noapo_220 = np.std(beta_pixels_qs_noapo_220)
std_beta_pixels_qs_apo_150 = np.std(beta_pixels_qs_apo_150)
std_beta_pixels_qs_apo_220 = np.std(beta_pixels_qs_apo_220)

std_beta_alms_fs_noapo_150 = np.std(beta_alms_fs_noapo_150)
std_beta_alms_fs_noapo_220 = np.std(beta_alms_fs_noapo_220)
std_beta_alms_qs_noapo_150 = np.std(beta_alms_qs_noapo_150)
std_beta_alms_qs_noapo_220 = np.std(beta_alms_qs_noapo_220)
std_beta_alms_qs_apo_150 = np.std(beta_alms_qs_apo_150)
std_beta_alms_qs_apo_220 = np.std(beta_alms_qs_apo_220)

In [ ]:
beta_pixels_fs_noapo_150

In [ ]:
print("For full sky without apo : ")
print("\n    Alms : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_fs_noapo_150, std_beta_alms_fs_noapo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_fs_noapo_220, std_beta_alms_fs_noapo_220))
print("\n    Pixels : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_fs_noapo_150, std_beta_pixels_fs_noapo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_fs_noapo_220, std_beta_pixels_fs_noapo_220))

print("\n For QUBIC sky without apo : ")
print("\n    Alms : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_qs_noapo_150, std_beta_alms_qs_noapo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_qs_noapo_220, std_beta_alms_qs_noapo_220))
print("\n    Pixels : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_qs_noapo_150, std_beta_pixels_qs_noapo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_qs_noapo_220, std_beta_pixels_qs_noapo_220))

print("\n For QUBIC sky with apo : ")
print("\n    Alms : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_qs_apo_150, std_beta_alms_qs_apo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_alms_qs_apo_220, std_beta_alms_qs_apo_220))
print("\n    Pixels : ")
print("\n        150 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_qs_apo_150, std_beta_pixels_qs_apo_150))
print("\n        220 GHz : {:.5f} +/- {:.12f}".format(mean_beta_pixels_qs_apo_220, std_beta_pixels_qs_apo_220))